In [1]:
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
def aceptar_cookies(año, liga, jornada):
    driver_dir = 'chrome_driver/chromedriver.exe'
    driver=webdriver.Chrome(driver_dir)
    driver.get("https://www.resultados-futbol.com/"+liga+str(año)+"/grupo1/jornada"+str(jornada))
    accept_cookies = driver.find_elements_by_xpath('//button[@class="sc-ifAKCX hYNOwJ"]')
    
    try:
        for button in accept_cookies:
            if button.text == "ACEPTO":
                relevant_button = button
                relevant_button.click()
    except:
        pass
    finally:
        return driver

In [3]:
def extraer_datos_jornada(soup):
    '''
    DOCSTRING
    '''
    table_soup = soup.find("table", {"id": 'tabla2'}).find('tbody').find_all('tr')
    num_equipos = len(table_soup)

    Posicion = [table_soup[i].find('th').text for i in range(num_equipos)]
    Equipo = [table_soup[i].find('td', {'class':'equipo'}).find('a').text for i in range(num_equipos)]
    Puntos = [table_soup[i].find('td', {'class':'pts'}).text for i in range(num_equipos)]
    Jornada = [table_soup[i].find('td', {'class':'pj'}).text for i in range(num_equipos)]
    Ganados = [table_soup[i].find('td', {'class':'win'}).text for i in range(num_equipos)]
    Empatados = [table_soup[i].find('td', {'class':'draw'}).text for i in range(num_equipos)]
    Perdidos = [table_soup[i].find('td', {'class':'lose'}).text for i in range(num_equipos)]
    Goles_a_favor= [table_soup[i].find('td', {'class':'f'}).text for i in range(num_equipos)]
    Goles_en_contra= [table_soup[i].find('td', {'class':'c'}).text for i in range(num_equipos)]

    tabla_jornada = [Posicion, Equipo, Puntos, Jornada, Ganados, Empatados, Perdidos, Goles_a_favor, Goles_en_contra]

    # Nos aseguramos que ningun dato se ha saltado, y que hemos encontrado todos
    if len(set([len(i) for i in tabla_jornada])) == 1:
        return tabla_jornada
    else:
        return [None] * len(tabla_jornada)

In [14]:
def extraer_datos_partido(soup):
    '''
    DOCSTRING
    '''
    regex = re.compile('vevent')
    
    table_soup = soup.find("table", {"id": 'tabla1'}).find('tbody').find_all("tr", {"class" : regex})
    num_partidos = len(table_soup)

    Equipo_Local = [table_soup[i].find('td', {'class':'equipo1'}).find_all('a')[1].text for i in range(num_partidos)]
    Equipo_Visitante = [table_soup[i].find('td', {'class':'equipo2'}).find_all('a')[1].text for i in range(num_partidos)]
    Resultado = [table_soup[i].find('td', {'class':'rstd'}).find('a', {'class':'url'}).find('span').text for i in range(num_partidos)]
    Fecha = [table_soup[i].find('td', {'class':'fecha'}).text for i in range(num_partidos)]

    tabla_partido = [Equipo_Local, Equipo_Visitante, Resultado, Fecha]

    # Nos aseguramos que ningun dato se ha saltado, y que hemos encontrado todos
    if len(set([len(i) for i in tabla_partido])) == 1:
        return tabla_partido
    else:
        return [None] * len(tabla_partido)

In [15]:
def extraer_num_jornadas(soup):
    jornadas_soup = soup.find("div", {"id":'desplega_jornadas', "class":'hidden'}).find_all('li')
    return len(jornadas_soup)

In [16]:
def extraccion_datos(Ano1, Ano2, *Ligas):
    '''
    DOCSTRING
    '''
    for liga in Ligas:
        for año in range(Ano1, Ano2):
            print(f'Extrayendo datos del año {año}') 
            driver = aceptar_cookies(año, liga, 1) # Accedemos simplemente a la primera jornada de ese año para comprobar cuantas jornadas hubo ese año
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
            num_jornadas = extraer_num_jornadas(soup)
            driver.quit()
            for jornada in range(1, num_jornadas + 1):
                print(f'\tExtrayendo jornada numero {jornada} del año {año}') 
                driver = aceptar_cookies(año, liga, jornada)
                page = driver.page_source
                soup = BeautifulSoup(page, 'html.parser')
                tabla_jornada = extraer_datos_jornada(soup)
                tabla_partido = extraer_datos_partido(soup)
                
                for i, key in enumerate(list_jornadas[:-2]):
                    dict_jornadas[key].extend(tabla_jornada[i])

                dict_jornadas['Año'].extend([año] * len(tabla_jornada[0]))
                dict_jornadas['Liga'].extend([liga] * len(tabla_jornada[0]))

                for i, key in enumerate(list_partidos[:-3]):
                    dict_partidos[key].extend(tabla_partido[i])

                dict_partidos['Año_partido'].extend([año] * len(tabla_partido[0]))
                dict_partidos['Jornada_partido'].extend([jornada] * len(tabla_partido[0]))
                dict_partidos['Liga_partido'].extend([liga] * len(tabla_partido[0]))
               
                print(f'\tDatos extraídos, quedan {num_jornadas - jornada} jornadas restantes este año')
                driver.quit()
            print(f'Datos del año {año} extraídos, quedan {Ano2 - año} años restantes') 

In [17]:
list_jornadas = ['Posicion', 'Equipo', 'Puntos', 'Jornada', 'Ganados', 'Empatados', 'Perdidos', 'Goles_a_favor', 'Goles_en_contra', 'Año', 'Liga'] 
list_partidos = ['Equipo_Local', 'Equipo_Visitante', 'Resultado', 'Fecha', 'Año_partido', 'Jornada_partido', 'Liga_partido']
dict_jornadas = {x:[] for x in list_jornadas}
dict_partidos = {x:[] for x in list_partidos}

In [18]:
extraccion_datos(1990, 1991, 'segunda')

Extrayendo datos del año 1990
	Extrayendo jornada numero 1 del año 1990
	Datos extraídos, quedan 37 jornadas restantes este año
	Extrayendo jornada numero 2 del año 1990
	Datos extraídos, quedan 36 jornadas restantes este año
	Extrayendo jornada numero 3 del año 1990
	Datos extraídos, quedan 35 jornadas restantes este año
	Extrayendo jornada numero 4 del año 1990
	Datos extraídos, quedan 34 jornadas restantes este año
	Extrayendo jornada numero 5 del año 1990
	Datos extraídos, quedan 33 jornadas restantes este año
	Extrayendo jornada numero 6 del año 1990
	Datos extraídos, quedan 32 jornadas restantes este año
	Extrayendo jornada numero 7 del año 1990


KeyboardInterrupt: 

In [21]:
Data_Tabla=pd.DataFrame(dict_jornadas)

In [23]:
Data_Tabla

,Posicion,Equipo,Puntos,Jornada,Ganados,Empatados,Perdidos,Goles_a_favor,Goles_en_contra,Año,Liga
0,1,Real Burgos CF,2,1,1,0,0,3,0,1990,segunda
1,2,Sabadell,2,1,1,0,0,1,0,1990,segunda
2,3,Bilbao Ath.,2,1,1,0,0,2,1,1990,segunda
3,4,Deportivo,2,1,1,0,0,1,0,1990,segunda
4,5,Xerez,2,1,1,0,0,4,1,1990,segunda
...,...,...,...,...,...,...,...,...,...,...,...
115,16,Real Murcia,4,6,1,2,3,4,7,1990,segunda
116,17,RM Castilla,4,6,0,4,2,3,7,1990,segunda
117,18,Recreativo,4,6,1,2,3,6,10,1990,segunda
118,19,Sestao Sport Cl.,3,6,0,3,3,1,5,1990,segunda


In [ ]:
Data_Tabla=pd.DataFrame([Posicion,Equipo,Puntos,Año,Jornada,Ganados,Empatados,Perdidos,Goles_a_favor,Goles_en_contra,Liga]).transpose()
Data_Tabla.columns=['Posicion','Equipo','Puntos','Año','Jornada','Ganados','Empatados','Perdidos','Goles_a_favor','Goles_en_contra','Liga']
Data_Tabla.to_excel("DDBB_Tabla_España_"+str(Ano1)+"_"+str(Ano2)+".xlsx") 

Data=pd.DataFrame([Equipo_Local,Equipo_Visitante,Resultado,Año_,Jornada_,Fecha,Liga_]).transpose()
Data.columns=['Equipo_Local','Equipo_Visitante','Resultado','Año','Jornada','Fecha','Liga']
Data.to_excel("DDBB_Resultados_España_"+str(Ano1)+"_"+str(Ano2)+".xlsx")  